In [29]:
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

In [3]:
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv("../../data/processed/data_train.csv")

In [ ]:
df

In [135]:
alcohol = ['Alcohol']

nicotine = ['Nicotine']

illegal_drugs1 = ['Cannabis', 'Legalh', 'LSD', 'Shrooms', 
                  'Amphet', 'Amyl', 'Ecstasy', 'Ketamine', 
                  'Coke']

illegal_drugs2 = ['Meth', 'Benzos', 'Heroin', 'Crack']

# SMOTE for imbalanced data

In [2]:
from imblearn.over_sampling import SMOTE

In [58]:
df.head()

,Gender,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Shrooms,Nicotine,VSA,Alcohol_bin,Amphet_bin,Amyl_bin,Benzos_bin,Caff_bin,Cannabis_bin,Choc_bin,Coke_bin,Crack_bin,Ecstasy_bin,Heroin_bin,Ketamine_bin,Legalh_bin,LSD_bin,Meth_bin,Shrooms_bin,Nicotine_bin,VSA_bin,age_25-34,age_35-44,age_45-54,age_55-64,age_65+,educ_at_17,educ_at_18,educ_before_16,educ_diploma,educ_doctorate_degree,educ_masters_degree,educ_some_college,educ_university_degree,country_canada,country_new_zealand,country_other,country_republic_of_ireland,country_uk,country_usa,eth_black,eth_black-asian,eth_other,eth_white,eth_white-asian,eth_white-black
0,0,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,5,2,0,2,6,0,5,0,0,0,0,0,0,0,0,0,2,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,1,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,5,2,2,0,6,4,6,3,0,4,0,2,0,2,3,0,4,0,1,1,1,0,1,1,1,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,1,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,6,0,0,0,6,3,4,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,0,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,4,0,0,3,5,2,4,2,0,0,0,2,0,0,0,0,2,0,1,0,0,1,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,0,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,4,1,1,0,6,3,6,0,0,1,0,0,1,0,0,2,2,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [59]:
df.Alcohol_bin.value_counts()/df.shape[0]

1    0.964305
0    0.035695
Name: Alcohol_bin, dtype: float64

In [117]:
features = df.columns.tolist()[:8] + df.columns.tolist()[44:]
X = df[features]
y = df['Alcohol_bin']
y

0       1
1       1
2       1
3       1
4       1
       ..
1872    1
1873    1
1874    1
1875    1
1876    1
Name: Alcohol_bin, Length: 1877, dtype: int64

In [118]:
Counter(y)

Counter({1: 1810, 0: 67})

In [119]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

pandas.core.series.Series

0       1
1       1
2       1
3       1
4       1
       ..
1872    1
1873    1
1874    1
1875    1
1876    1
Name: Alcohol_bin, Length: 1877, dtype: int64

In [105]:
clf_A = DecisionTreeClassifier(random_state=101)
clf_B = SVC(random_state = 101)
clf_C = AdaBoostClassifier(random_state = 101)

#Define Parameters
parameters_DT = {'max_depth': np.arange(1, 101, 1),             
             } 

classifieres = [clf_A]

In [106]:
def train_predict(clf, parameters, drugs, X, y,cv=5):
    results = {}
    # Choose the clf and define paramters
  
    #Make the scorer
    scores = [
        'precision', 
        'recall', 
       # 'accuracy', 
        'f1', 
       # 'roc_auc'
    ]
    
    # Perform GridSearch
    best_scores = {}
    best_param = {}
    #best_model = {}
    for d in drugs:
        #X_train, X_test, y_train, y_test = train_test_split(X, df[y], test_size=0.2, random_state=1, stratify=df[y])
        grid = GridSearchCV(estimator=clf, param_grid=parameters, 
                            cv=cv, scoring=scores, refit='recall')
        grid_fit = grid.fit(X,y[d]) 
        best_scores[d] = {}
        best_scores[d]['recall'] = max(grid_fit.cv_results_['mean_test_recall'])
        best_scores[d]['precision'] = max(grid_fit.cv_results_['mean_test_precision'])
        best_param[d] = grid_fit.best_params_
        best_clf = grid_fit.best_estimator_ 
        
    return best_scores, best_param

In [107]:
best_scores, best_param = train_predict(clf_A, parameters_DT, 
                                        ['Alcohol_bin', 'Meth_bin'], X, y,cv=4)

/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [108]:
best_scores

{'Alcohol_bin': {'recall': 0.9944812362030905, 'precision': 0.965553045632532},
 'Meth_bin': {'recall': 0.5969940253920836, 'precision': 0.5957954482515334}}

In [104]:
best_param

{'Alcohol_bin': {'max_depth': 2}, 'Meth_bin': {'max_depth': 2}}

# Plot ROC curve and confusion matrix